In [1]:
import loompy
import os
import pandas as pd
import numpy as np
import re
import gc

/home/owenwhitley/anaconda3/envs/scvelo_conda/lib/python3.8/site-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9002. The TBB threading layer is disabled.
  warnings.warn(problem)


In [2]:
rerun_merge = True

In [3]:
top_dir = "/home/owenwhitley/projects/su2c_v2"
inp_data_dir = os.path.join(top_dir, "data/raw/scRNA/Data/velocyto_GBM_GSCs_split_by_sample_no_G800_L")
output_dir = os.path.join(top_dir, 'data/preprocessed/scRNA/velocyto_GBM_GSCs_split_by_sample_no_G800_L')
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
fnames = os.listdir(inp_data_dir)
fnames

['G885_L.loom',
 'G1003-D_T.loom',
 'G946-K_T.loom',
 'G946-K_L.loom',
 'G910-C_T.loom',
 'G620_T.loom',
 'G1003-C_T.loom',
 'G549_L.loom',
 'BT127_L.loom',
 'G945-J_T.loom',
 'G910-D_T.loom',
 'BT73_L.loom',
 'G851_L.loom',
 'G895_L.loom',
 'G583_L.loom',
 'G945-J_L.loom',
 'G637_L.loom',
 'G945-K_T.loom',
 'G946-J_T.loom',
 'G876_L.loom',
 'G910-B_T.loom',
 'BT48_L.loom',
 'G967-A_T.loom',
 'G729_L.loom',
 'G967-D_T.loom',
 'BT94_L.loom',
 'G983-A_T.loom',
 'G797_L.loom',
 'G1003-B_T.loom',
 'G620_L.loom',
 'G945-K_L.loom',
 'G946-J_L.loom',
 'G837_L.loom',
 'G983-C_T.loom',
 'BT89_L.loom',
 'BT147_L.loom',
 'G983-B_T.loom',
 'G945-I_T.loom',
 'G799_L.loom',
 'G910-A_T.loom',
 'BT84_L.loom',
 'G523_L.loom',
 'G910-E_T.loom',
 'G1003-A_T.loom',
 'G967-B_T.loom',
 'G967-C_T.loom',
 'BT67_L.loom',
 'G946-I_T.loom',
 'G566_L.loom',
 'G564_L.loom',
 'G945-I_L.loom']

In [4]:
def find_duplicates(arr):
    unique_vals = np.unique(arr)
    val_dict = {}
    is_duplicated = []
    for i in range(len(arr)):
        val = arr[i]
        in_dict = val in val_dict.keys()
        if in_dict:
            return_val = True
        else:
            return_val = False
            val_dict[val] = True
        is_duplicated.append(return_val)
        
    return(np.array(is_duplicated))

In [5]:
# fnames = ['G620_L.loom', 'G620_T.loom']
n = len(fnames)
fpath_list = [os.path.join(inp_data_dir, x) for x in fnames[0:n]]


out_file = os.path.join(output_dir, 'GBM_GSCs_combined.loom')
if rerun_merge:
    has_duplicates = []
    new_file_list = []
    for f in fpath_list:
        loom_con = loompy.connect(f)
        cell_ids = loom_con.ca['CellID']
        prefix = re.search('(G|BT)[0-9]*[-]*[A-Z]*_[TL]', os.path.basename(f)).group(0)
        barcodes = [re.search('[ATCG]*x$', x).group(0) for x in cell_ids]
        barcodes = [re.search('[ATCG]*', x).group(0) for x in barcodes]
        barcodes = np.array([prefix + '_' + x for x in barcodes])
        has_duplicates.append(np.any(find_duplicates(barcodes)))
        new_file = os.path.join(output_dir, prefix + '_renamed.loom')
        if os.path.exists(new_file):
            os.remove(new_file)
        loompy.create(new_file, loom_con.layers, loom_con.ra, loom_con.ca)
        loom_con.close()
        loom_new = loompy.connect(new_file)
        loom_new.ca['CellID'] = barcodes
        loom_new.close()
        new_file_list.append(new_file)
    
    if os.path.exists(out_file):
        os.remove(out_file)
    loompy.combine(new_file_list, out_file, convert_attrs = 'Accession')
    
    # edit loom file unique ids to match plot data unique ids
    output_con = loompy.connect(out_file)
    loom_cell_ids = output_con.ca['CellID']
    loom_cell_ids = np.array(loom_cell_ids)
#     tumour_regex = '_T_'
#     line_regex = '_L_'

#     new_loom_ids = []

#     for x in loom_cell_ids:
#         new_id = ''
#         if re.search(tumour_regex, x):
#             new_id = 'GBM_' + x
#         elif re.search(line_regex, x):
#             new_id = 'BTSC_' + x
#         else:
#             raise ValueError()
#         new_loom_ids.append(new_id)
        
#     if np.any(find_duplicates(new_loom_ids)):
#         raise ValueError('duplicate new loom ids')
    
#     output_con.ca['CellID'] = new_loom_ids
    output_con.close()
        
    